In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import math
from tqdm import tqdm
import Util as ut
# import seaborn as sns
import DataLoader
import traceback
import pandas as pd

%matplotlib inline

import PeakSearcher

indir = '/Users/yhamae/OneDrive/astro/FLASHING/spectrum/'
outdir = '/Users/yhamae/OneDrive/astro/FLASHING/train/'
ref_freq_list = ['H2O', 'SiOv3', 'SiOv2', 'SiOv1', 'SiOv0']

err_file = []

source_list = []
file_list = os.listdir(indir)
os.chdir(indir)

for fname in file_list:
    if not fname.split('_')[0] in [source_list[i][0] for i in range(0, len(source_list))]:
        for val in ref_freq_list:
            source_list.append([fname.split('_')[0], val])

for lis1 in tqdm(source_list):
#     print(lis1)
    in_file_list = []
    data = {}
    for fl in file_list:
        if lis1[0] == fl.split('_')[0] and lis1[1] == fl.split('_')[1]:
            in_file_list.append(fl)
    if len(in_file_list) == 0:
        continue
    
    for read_fname in in_file_list:
        dl = DataLoader.GetSpectrum()
        dl.filename = read_fname
        dl.get_data()
        if dl.MJD == 'N/A':
            continue
        data[float(dl.MJD)] = []
        for channel, freq, T in zip(dl.channel, dl.freq, dl.T):
            data[dl.MJD].append([float(channel), float(freq), float(T)])
    check_freq = []
    if len(data.keys()) == 1:
        continue
#     else:
#         check_freq = [lis[1] for lis in data[list(data.keys())[0]]]
#     for key in list(data.keys())[1:]:
#         for i, f in enumerate(data[key]):
#             if f[1] != check_freq[i]:
#                 print('LSR V is not match!!')
    exp_data = []
#     print(data.keys())
    for i in range(0, 2048):
#         tmp_lis = [check_freq[i]]
        tmp_lis = []
        for key in data.keys():
#             tmp_lis.apabspend(data[key][i][2])
            if len(data[key]) > i:
                tmp_lis.extend(data[key][i][1:3])
            else:
                tmp_lis.extend([0,0])
        exp_data.append(tmp_lis)
#     print(exp_data)
#     col = ['LSR Velocity(km/s)']
    col = []
#     col.extend(data.keys())
    for key in data.keys():
        col.extend([str(key) + '_LSR_v', str(key) + ''])
    df = pd.DataFrame(exp_data, columns = col)
    df.to_csv(outdir + lis1[0] + '_' + lis1[1] + '.csv')
    

100%|██████████| 445/445 [00:36<00:00, 12.05it/s]
